<a href="https://colab.research.google.com/github/Ziqi-Li/GIS5122/blob/main/notebooks/W10_Multi_level_Model_varying_intercept_(Colab).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Multi-level models (Varying Intercept)
***
This is a workaround to run the model in Google Colab. However, installing the packages every time would be a little inconvience and take extra time.

#### Installation


In [1]:
pip install -q pymer4==0.7.8

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.0/59.0 kB 1.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.4/194.4 kB 5.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done


In [2]:
import rpy2.ipython
%load_ext rpy2.ipython

import logging
# Suppress Python warnings for rpy2 callbacks
logging.getLogger('rpy2.rinterface_lib.callbacks').setLevel(logging.ERROR)

In [3]:
%%time
#This may take a while (~5 mins)
%R install.packages("lme4")
%R install.packages("lmerTest");

CPU times: user 3.82 s, sys: 354 ms, total: 4.18 s
Wall time: 5min 14s


Load packages

In [4]:
import numpy as np
import matplotlib.pyplot as plt
import geopandas as gpd
import pandas as pd

In [5]:
from pymer4.models import Lmer

We will be using the same voting data to demostrate the MLM. The data is slightly different in the fact that it has an extra column indicating the governer of the state in which a county belongs to.

Load voting dataset

In [6]:
voting = pd.read_csv('https://raw.github.com/Ziqi-Li/gis5122/master/data/voting_2020_with_gov.csv')

voting['is_dem_gov']= (voting.party == 'democrat')

In [7]:
shp = gpd.read_file("https://raw.github.com/Ziqi-Li/gis5122/master/data/us_counties.geojson")

In [8]:
#Merge the shapefile with the voting data by the common county_id
shp_voting = shp.merge(voting, on ="county_id")

shp_voting_df = shp_voting.drop(columns='geometry')

#Dissolve the counties to obtain boundary of states, used for mapping
state = shp_voting.dissolve(by='STATEFP').geometry.boundary

### Fit an MLM model (Varying intercept by state)

Let us first look at a pair wise comaprison between

- California vs. Massachusetts
- California vs. Texas

Note that the State's FIPS code is:

CA:06;MA:25;TX:48

In [9]:
ca_ma = shp_voting_df[shp_voting_df['state'].isin([6, 25])]

ca_tx = shp_voting_df[shp_voting_df['state'].isin([6, 48])]

The `Lmer()` takes a formula where follows the format `y ~ X`. 1 indicates adding an intercept, and `(1|state)` allowing the intercept to vary across states.

California vs. Texas

In [10]:
from pymer4.models import Lmer

model = Lmer('new_pct_dem ~ 1 + (1|state)', data=ca_tx)

model.fit()

/usr/local/lib/python3.10/dist-packages/rpy2/robjects/pandas2ri.py:55: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for name, values in obj.iteritems():


Formula: new_pct_dem~1+(1|state)

Family: gaussian	 Inference: parametric

Number of observations: 312	 Groups: {'state': 2.0}

Log-likelihood: -1278.940 	 AIC: 2557.881

Random effects:

                 Name      Var     Std
state     (Intercept)  458.854  21.421
Residual               210.864  14.521

No random effect correlations specified

Fixed effects:



,Estimate,2.5_ci,97.5_ci,SE,DF,T-stat,P-val,Sig
(Intercept),39.981,10.222,69.74,15.184,1.0,2.633,0.231,


From the summary:

- There are 312 observations (total number of counties in CA + total number of counties in TX)
- There are 2 groups
- The model has an AIC of 2557.881
- The state-lavel random effect has an estimated variance of 458.854 with a Std of 21.421
- The residual has an estimated variance 210.864 with a std of 14.521.
- The Variance Partition Coefficient (VPC) = 458.854/(458.854+210.864) = 68.5%. This means 68.5% of the variance in the voting preference can be attributed to the fact that counties belong to two seperate states.
- The interpretation to the fixed effects are the same as we do in the linear regression. There are no predictors, only one intercept here, and the intercept estimate is 39.981, and it is not statistically significance (p-value = 0.231 > 0.05).



California vs. Mass.

In [11]:
model2 = Lmer('new_pct_dem ~ 1 + (1|state)', data=ca_ma)

model2.fit()

/usr/local/lib/python3.10/dist-packages/rpy2/robjects/pandas2ri.py:55: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for name, values in obj.iteritems():


Formula: new_pct_dem~1+(1|state)

Family: gaussian	 Inference: parametric

Number of observations: 72	 Groups: {'state': 2.0}

Log-likelihood: -293.063 	 AIC: 586.127

Random effects:

                 Name      Var     Std
state     (Intercept)   77.881   8.825
Residual               205.493  14.335

No random effect correlations specified

Fixed effects:



,Estimate,2.5_ci,97.5_ci,SE,DF,T-stat,P-val,Sig
(Intercept),61.384,48.484,74.284,6.582,0.984,9.327,0.07,.


### Varying intercept model for all states

In [12]:
model_all_states_1 = Lmer('new_pct_dem ~ 1 + (1|state)', data=shp_voting_df)

model_all_states_1.fit()

/usr/local/lib/python3.10/dist-packages/rpy2/robjects/pandas2ri.py:55: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for name, values in obj.iteritems():


Formula: new_pct_dem~1+(1|state)

Family: gaussian	 Inference: parametric

Number of observations: 3102	 Groups: {'state': 47.0}

Log-likelihood: -12510.156 	 AIC: 25020.312

Random effects:

                 Name      Var     Std
state     (Intercept)  134.233  11.586
Residual               176.567  13.288

No random effect correlations specified

Fixed effects:



,Estimate,2.5_ci,97.5_ci,SE,DF,T-stat,P-val,Sig
(Intercept),37.507,34.129,40.885,1.724,43.861,21.761,0.0,***


What is VPC in this case?

### Now we add some county-level (individual) variable

Here we include `pct_bach` in the model as a fixed effect

In [13]:
model_all_states_2 = Lmer('new_pct_dem ~ 1 + pct_bach + (1|state)', data=shp_voting_df)

model_all_states_2.fit()

/usr/local/lib/python3.10/dist-packages/rpy2/robjects/pandas2ri.py:55: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for name, values in obj.iteritems():


Formula: new_pct_dem~1+pct_bach+(1|state)

Family: gaussian	 Inference: parametric

Number of observations: 3102	 Groups: {'state': 47.0}

Log-likelihood: -12069.178 	 AIC: 24138.356

Random effects:

                 Name      Var     Std
state     (Intercept)   76.666   8.756
Residual               133.186  11.541

No random effect correlations specified

Fixed effects:



,Estimate,2.5_ci,97.5_ci,SE,DF,T-stat,P-val,Sig
(Intercept),18.930,16.125,21.735,1.431,62.434,13.226,0.0,***
pct_bach,0.776,0.728,0.823,0.024,3097.039,32.086,0.0,***


Model AIC decreases as adding this `pct_bach` variable. The interpretation to this variable is: After accounting for the state-level differences, the `pct_bach` has an estimate of 0.776 with a low p-value. Nationally, increasing 1% in `pct_bach` will result in increase `0.776%` people voted for the DEM, while keep other factors constant.

### Now we add a state-level (group) variable

Here we include `is_dem_gov` in the model as a fixed effect

In [14]:
model_all_states_3 = Lmer('new_pct_dem ~ 1 + pct_bach + is_dem_gov + (1|state)', data=shp_voting_df)

model_all_states_3.fit()

/usr/local/lib/python3.10/dist-packages/rpy2/robjects/pandas2ri.py:55: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for name, values in obj.iteritems():


Formula: new_pct_dem~1+pct_bach+is_dem_gov+(1|state)

Family: gaussian	 Inference: parametric

Number of observations: 3102	 Groups: {'state': 47.0}

Log-likelihood: -12065.543 	 AIC: 24131.087

Random effects:

                 Name      Var     Std
state     (Intercept)   72.165   8.495
Residual               133.190  11.541

No random effect correlations specified

Fixed effects:



,Estimate,2.5_ci,97.5_ci,SE,DF,T-stat,P-val,Sig
(Intercept),16.666,13.087,20.244,1.826,51.528,9.127,0.000,***
pct_bach,0.775,0.728,0.823,0.024,3096.963,32.081,0.000,***
is_dem_govTRUE,4.864,-0.138,9.867,2.552,43.097,1.906,0.063,.


After accounting for the state-level differences, the `is_dem_gov` has an estimate of 4.864 with a high p-value (0.063 > 0.05). This is saying that nationally, if a state has a democratic governer will contribute to 4.8% percent more vote into DEM, however, this effect is not statistically significant at 0.05 level given the current data and model.

In [15]:
shp_voting_df.columns

Index(['GEOID', 'STATEFP', 'NAME_x', 'county_id', 'Unnamed: 0', 'state',
       'county', 'NAME_y', 'proj_X', 'proj_Y', 'total_pop', 'new_pct_dem',
       'sex_ratio', 'pct_black', 'pct_hisp', 'pct_bach', 'median_income',
       'pct_65_over', 'pct_age_18_29', 'gini', 'pct_manuf', 'ln_pop_den',
       'pct_3rd_party', 'turn_out', 'pct_fb', 'pct_uninsured', 'stname', ' st',
       ' stusps', 'state_code', 'party', 'is_dem_gov'],
      dtype='object')